In [1]:
import pandas as pd
import numpy as np
import os
import re
import random
import pandas as pd

# Loading Product Review Dataset

In [2]:
df = pd.read_csv("aws_review_sofware_dataset.csv")

C:\Users\91798\AppData\Local\Temp\ipykernel_15320\2031363380.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("aws_review_sofware_dataset.csv")


In [3]:
df.columns

Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')

# Take only few rows and columns

In [4]:
df = df.sample(n=10000,random_state = 42)
df = df[df.index.isin(df.index)]
df = df.reset_index(drop = True)

In [5]:
df

,Unnamed: 0,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,359583,1.0,True,"04 30, 2016",A3K6NXE4QDCLIG,B018OJLIX6,NaN,Kent,We run a top of the line system utilizing Wind...,Does not work on Windows 10,1461974400,6,NaN
1,141722,5.0,True,"11 14, 2015",AXFINZCCNA8NV,B008H3SW4I,{'Format:': ' Software'},Amazon Customer,I don't really know why there were so many com...,Works well for me,1447459200,NaN,NaN
2,94354,5.0,True,"01 20, 2011",A1FWHS8Q7G8VKJ,B0039Y58YC,NaN,Solitary,I have used both WordPerfect Office and Micros...,Software is User Friendly,1295481600,NaN,NaN
3,215474,4.0,True,"09 10, 2016",A1MY6QNNH340MX,B00F3ZN2W0,NaN,Numb,"If you are buying for a build, don't be like m...",don't be like me and make sure you either have...,1473465600,NaN,NaN
4,367254,1.0,True,"03 25, 2016",A1F2KSFD73HHG6,B00001QGWI,NaN,Davis V Hill,Did not meet expectations. Too juvenile for m...,Be careful!,1458864000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,166882,4.0,False,"09 22, 2013",A198C37V3NQZI0,B00B1JWX2M,NaN,TJ Williams,I ordered Proposal Pack Wizard in 2011 and the...,A Great Product,1379808000,NaN,NaN
9996,171707,5.0,True,"01 14, 2015",AD4F0J7JO5NM,B00BBJPPXG,{'Platform:': ' PC/Mac/Android/iOS Code'},Francois P.,i've been using Norton 360 with windows 7 for ...,and i am satisfied.,1421193600,NaN,NaN
9997,398604,4.0,True,"12 26, 2007",A2Y0TX9P5GIHRY,B000I84DSY,NaN,Darrel,Very good protection and does not bog down the...,CA Internet Security Suite 2007 - 3 User,1198627200,NaN,NaN
9998,424597,3.0,True,"03 14, 2015",A38O8TX78RPDOI,B005FIWU3W,{'Format:': ' CD-ROM'},smartypants,"Quickbooks is a great product. HOWEVER, the f...",Does what it's suppose to.,1426291200,NaN,NaN


# Checking for Null Values

In [6]:
df.isnull().sum()

Unnamed: 0           0
overall              0
verified             0
reviewTime           0
reviewerID           0
asin                 0
style             4930
reviewerName         1
reviewText           0
summary              1
unixReviewTime       0
vote              7143
image             9965
dtype: int64

In [7]:
df["words"]="default value"
df["sentences"]="default value"


for i in range(df.shape[0]):
    df.at[i,"words"]= list("")
    df.at[i,"sentences"] = list("")

# Sentence Tokenization

In [8]:
from nltk.tokenize import sent_tokenize

In [9]:
for i in range(10000):
    l1= sent_tokenize(df.loc[i,"reviewText"])
    df.at[i,"sentences"]=l1

# Word Tokenization

In [10]:
from pywsd.utils import lemmatize_sentence

Warming up PyWSD (takes ~10 secs)... took 3.430290937423706 secs.


In [11]:
for k in range(10000):
    df.at[k,"words"]=list("")
    for j in range(len(df.loc[k,"sentences"])):
        df.at[k,"words"].extend(lemmatize_sentence(df.loc[k,"sentences"][j]))

In [12]:
df["words_sentences"] = [' ' for i in range(10000)]

In [13]:
def get_str(list1):
    s1 = ''
    for i1 in list1:
        s1 = s1+' '+i1
    return s1

In [14]:
import functools
for k in range(df.shape[0]):
    df.loc[k,"words_sentences"]=get_str(df.loc[k,"words"])

# TF-IDF Vectorizer

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df.words_sentences)

In [17]:
dense = pd.DataFrame(tfidf_matrix.todense(), columns=tfidf_vectorizer.get_feature_names_out())

In [18]:
dense.head()

,00,000,000000000003,00045,000articles,000yrs,001,008866,009,00gb,...,zookeeper,zoom,zoombini,zoombiniland,zoombinis,zooming,zr10,zu3d,zunko,zuzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df_y=df["verified"]

# Label Encoding

In [20]:
from sklearn.preprocessing import LabelEncoder

In [21]:
le=LabelEncoder()

In [22]:
df_y_1=pd.DataFrame(df_y)

In [23]:
df_y_enc=df_y_1.apply(le.fit_transform)

# Decision Tree Classifier

In [24]:
from sklearn.tree import DecisionTreeClassifier

In [25]:
DecTree=DecisionTreeClassifier()

In [26]:
dt_M=DecTree.fit(dense,df_y_enc)

In [27]:
dt_M.score(dense,df_y_enc)

0.9969